In [18]:
#These are the required libraries. To install them, you can uncomment the following lines and run the script.
#%pip install Pillow
#%pip install opencv-python
#%pip install keras
#%pip install tensorflow
#%pip install mtcnn

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from keras.models import load_model
from mtcnn.mtcnn import MTCNN

In [19]:
model = load_model('RoboticsExpressionModel.h5')
detector = MTCNN()

root = tk.Tk()
root.title("Facial Expression Recognition")

''

In [20]:
emotion_colors = {
    'angry': '#FF5733',
    'happy': '#FFFF33',
    'neutral': '#33FF57',
    'sad': '#335EFF',
    'surprised': '#F033FF'
}

#configure frames
container = tk.Frame(root)
container.pack(padx=20, pady=20, fill="both", expand=True)
progress_frame = tk.Frame(root)
progress_frame.pack(pady=20)

In [21]:
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image_rgb = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        display_image(image_rgb)
        detect_and_predict(image_rgb)

#function displaying the image
def display_image(image_rgb):
    image = Image.fromarray(image_rgb)
    image.thumbnail((750, 500), Image.Resampling.LANCZOS)
    image_tk = ImageTk.PhotoImage(image)
    image_label.configure(image=image_tk)
    image_label.image = image_tk

#function detecting faces and predicting emotions
def detect_and_predict(image_rgb):
    faces = detector.detect_faces(image_rgb)
    for face in faces:
        x, y, width, height = face['box']
        cv2.rectangle(image_rgb, (x, y), (x + width, y + height), (0, 155, 255), 2)
        for keypoint in face['keypoints'].values():
            cv2.circle(image_rgb, keypoint, 2, (0, 155, 255), 2)
    display_image(image_rgb)
    if faces:
        x, y, width, height = faces[0]['box']
        face_roi = image_rgb[y:y + height, x:x + width]
        predict_emotions(face_roi)

#function preprocessing the face and showing predictions
def predict_emotions(face_roi):
    face_resized = cv2.resize(face_roi, (112, 112)) / 255.0
    face_array = np.reshape(face_resized, (1, 112, 112, 3))
    predictions = model.predict(face_array)[0]
    show_predictions(predictions)

#function displaying prediction results
def show_predictions(predictions):
    class_names = ['angry', 'happy', 'neutral', 'sad', 'surprise']
    for widget in progress_frame.winfo_children():
        widget.destroy()
    for idx in np.argsort(predictions)[::-1]:
        emotion = class_names[idx]
        confidence = predictions[idx]
        tk.Label(progress_frame, text=f"{emotion}: {confidence * 100:.2f}%").pack(anchor='w')
        canvas = tk.Canvas(progress_frame, width=200, height=20)
        canvas.pack()
        fill_width = confidence * 200
        canvas.create_rectangle(0, 0, fill_width, 20, fill=emotion_colors.get(emotion, "#FFFFFF"))
        canvas.create_rectangle(fill_width, 0, 200, 20, fill="#D3D3D3")
        
#UI Elements
image_label = tk.Label(root)
image_label.pack(padx=20, pady=20)

upload_button = tk.Button(container, text="Upload an Image", command=upload_image,
                          font=('Helvetica', 12, 'bold'), fg='white', bg='#333333',
                          borderwidth=2, relief="raised", padx=10, pady=5)
upload_button.pack(pady=(10, 20))

root.mainloop()

1/1 [==============================] - 0s 16ms/step
